In [1]:
from datasets import load_dataset
import pandas as pd

/home/jaykold/dev/llm-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
full_data = load_dataset("squad_v2", split="train")

'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /datasets/squad_v2/resolve/main/README.md (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fc1fb557230>: Failed to resolve \'huggingface.co\' ([Errno -3] Temporary failure in name resolution)"))'), '(Request ID: 4f133381-de0c-40c7-b529-ae141cec521b)')' thrown while requesting HEAD https://huggingface.co/datasets/squad_v2/resolve/main/README.md
Retrying in 1s [Retry 1/5].
Generating validation split: 100%|██████████| 11873/11873 [00:00<00:00, 193735.64 examples/s]


In [3]:
shuffled_data = full_data.shuffle(seed=42)

type(shuffled_data)

datasets.arrow_dataset.Dataset

In [4]:
shuffled_data = pd.DataFrame(shuffled_data)

In [5]:
shuffled_data.head()

,id,title,context,question,answers
0,56e0f3907aa994140058e80a,Canon_law,The Roman Catholic Church canon law also inclu...,What term characterizes the intersection of th...,"{'text': ['full union'], 'answer_start': [104]}"
1,571adcf932177014007e9f56,Athanasius_of_Alexandria,Alexandria was the most important trade center...,What was Alexandria known for?,"{'text': ['important trade center'], 'answer_s..."
2,57325b9fe99e3014001e670c,Jehovah%27s_Witnesses,Former members Heather and Gary Botting compar...,How do the leaders of the Jehovah's Witnesses ...,{'text': ['disparaging individual decision-mak...
3,5728d8be4b864d1900164f6b,Estonia,"Historically, the cuisine of Estonia has been ...",What are the most common foods in Estonia?,"{'text': ['black bread, pork, potatoes, and da..."
4,56f6f5e1711bf01900a44898,Classical_music,Many of the instruments used to perform mediev...,What was the medieval flute made from?,"{'text': ['wood'], 'answer_start': [126]}"


In [16]:
#shuffled_data.to_csv("data/squad_v2.csv")

In [7]:
data = shuffled_data.head(500)

In [8]:
title = data.iloc[90]['title']

sx = data[data['title'] == title]['answers']

In [12]:
data['title']
df = pd.DataFrame(data)

In [13]:
df['title']

0                     Canon_law
1      Athanasius_of_Alexandria
2         Jehovah%27s_Witnesses
3                       Estonia
4               Classical_music
                 ...           
495       Alexander_Graham_Bell
496          Chinese_characters
497               War_on_Terror
498              Roman_Republic
499      Professional_wrestling
Name: title, Length: 500, dtype: object

In [14]:
df['title'].nunique()

285

In [15]:
df.sample(5)

,id,title,context,question,answers
313,57262bce89a1e219009ac4f5,Royal_Dutch_Shell,"Through most of Shell's early history, the She...",In what year did Royal Dutch Shell make a bid ...,"{'text': ['1984'], 'answer_start': [282]}"
362,5ace430732bba1001ae4a10c,Madrasa,The Arabic term ijāzat al-tadrīs was awarded t...,Who awarded a teaching license in non-Islamic...,"{'text': [], 'answer_start': []}"
224,57096178ed30961900e8402f,Himachal_Pradesh,The history of the area that now constitutes H...,When did Mahmud Ghanznavi conquer Kangara?,"{'text': ['10th century'], 'answer_start': [759]}"
139,5ace9ee632bba1001ae4ad6e,Brigham_Young_University,"BYU accepted 49 percent of the 11,423 people w...",What did BYU have 28 of in 2006?,"{'text': [], 'answer_start': []}"
414,570d98dddf2f5219002ed006,Anti-aircraft_warfare,The Germans developed massive reinforced concr...,What was placed on the blockhouses by the Germ...,"{'text': ['anti-aircraft artillery'], 'answer_..."
